# 1. Import Python modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import PIL
import pathlib
import random

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, f1_score, recall_score

import os
from os.path import exists

from utilsData import *
from utilsModel import *

# Configs

In [2]:
# folders to save NN model to file
model_path = "models/"
if not exists(model_path):
    os.mkdir(model_path)
    
# folder with datasets
data_path = "data/"

# name of the scenario
model_name = "cifar10_auto_truck" #"mnist_1_7", "mnist_3_8", "mnist_2_5","pneumoniaMNIST", "art"

delta = 1E-3
epsilon = 0.01

IMAGE_CHANNELS = 3 # 3 for rgb images

labels_dict = {0: "Automobile", 1: "Truck"} # "mnist_1_7": {0:"7",1:"1"}, "mnist_3_8": {0:"3",1:"8}", "mnist_2_5": {0:"2",1:"5"},"pneumoniaMNIST":{0:"normal",1:"pneumonia"}, "art": {0:"renaissance", 1:"baroque"}


# 2. Load dataset

In [3]:
X = np.load(data_path + model_name +"_images.npy")
Y = np.load(data_path + model_name +"_labels.npy")


# 3. Train Test split and save

In [29]:
X_train, Y_train, X_cal, Y_cal, X_test, Y_test = SplitDataset(X, Y, delta, epsilon)

print('Training set size: ', len(Y_train))
print('Test set size: ', len(Y_test))
print('Calibration set size: ', len(Y_cal))

Training set size:  1839
Test set size:  3000
Calibration set size:  5161


In [31]:
np.save(data_path + model_name+"_trainX.npy",X_train)
np.save(data_path + model_name+"_trainY.npy",Y_train)
np.save(data_path + model_name+"_testX.npy",X_test)
np.save(data_path + model_name+"_testY.npy",Y_test)
np.save(data_path + model_name+"_calX.npy",X_cal)
np.save(data_path + model_name+"_calY.npy",Y_cal)


# 4. Build convolutional neural network and train it

## 4.1 CNN architecture

In [32]:
IMAGE_WIDTH = X_train.shape[1]
IMAGE_HEIGHT = X_train.shape[2]

image_shape = (IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS)

model = CNN_Model(image_shape)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 batch_normalization_15 (Ba  (None, 30, 30, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 15, 15, 32)        0         
                                                                 
 conv2d_12 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 batch_normalization_16 (Ba  (None, 13, 13, 64)       

## 4.2 Training

In [33]:
history=model.fit(X_train, Y_train, epochs=3)

Epoch 1/3
58/58 [==============================] - 3s 16ms/step - loss: 0.8051 - accuracy: 0.6264
Epoch 2/3
58/58 [==============================] - 1s 16ms/step - loss: 0.6132 - accuracy: 0.7238
Epoch 3/3
58/58 [==============================] - 1s 16ms/step - loss: 0.5338 - accuracy: 0.7564


## 4.3 Save the model

In [34]:

model.save(f"{model_path}{model_name}.keras")
